# 🤖 微調實戰課程-P3-GRPO

> 使用 Trl 訓練框架、GRPO 策略微調 0.5B 模型，專屬於你的第一堂微調語言模型實戰課程
> 日期：2025/09/16-1020

<a href="https://www.tenlong.com.tw/products/9786264142915"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>

## 👨‍💻 作者資源與聯絡方式

### 📚 深度學習專書
**📖 《LangGraph 實戰開發 AI Agent 全攻略》** - 我的最新技術著作
深入探討 LangGraph、Agentic AI System 等前沿技術
**[立即購買](https://www.tenlong.com.tw/products/9786264142915)**

### 🌐 社群媒體與技術交流
如果您有任何疑問或想要進一步交流，歡迎透過以下管道聯絡：

* **📖 技術專書**： [購買我的 LangGraph 實戰開發 AI Agent 全攻略](https://www.tenlong.com.tw/products/9786264142915)
* **💻 GitHub**： [我的開源專案](https://github.com/Heng-xiu)
* **🤗 Hugging Face**： [我的模型與資料集](https://huggingface.co/Heng666)
* **✍️ 部落格**： [技術文章分享](https://r23456999.medium.com/)

感謝大家的支持！期待與更多 AI 技術愛好者交流討論 🚀

<div class="align-center">
  <a href="https://ko-fi.com/hengshiousheu"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a>
</div>

<a href="https://www.tenlong.com.tw/products/9786264142915"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>



# 第一章、可驗證獎勵強化學習
可驗證獎勵強化學習 (Reinforcement LEarning Verfied Reward; RLVR) 是一種訓練 AI 系統（尤其是大型語言模型）的方法，它根據輸出是否符合預先定義的正確性標準，為系統提供清晰客觀的回饋。與依賴主觀人工評估或複雜的學習獎勵模型的傳統強化學習方法不同，RLVR 使用基於規則的函數來評估模型反應的準確性，並提供二元獎勵：1 表示正確輸出，0 表示錯誤輸出。

---

#第二章、介紹 GRPO

GRPO（Group Relative Policy Optimization） 是一個相對新穎的做法。它的特點在於能夠「直接優化模型的偏好對齊」，換句話說，就是讓模型更快、更穩定地學會產生符合人類期待的輸出。相比傳統的 PPO，GRPO 的設計更簡潔，但效果往往更好。下面我們從三個核心元素來理解它的運作。

##2.1群組取樣（Group Sampling）：一次嘗試多種答案

在 GRPO 的第一步是針對每個問題產生多個可能的答案。這樣就能建立一組多樣化的輸出，之後可以彼此比較。模型不會只給出單一解答，而是針對同一個問題產生多個版本。常見的數量是 4、8，甚至 16 個。這有點像同一組學生被分派一樣的數學題，每個人都交出自己的解法。

有些解法會按部就班，先算總數再細分；有些解法可能直接跳步，或者方法雖不同卻同樣正確；當然，也會出現錯誤或比較笨拙的做法。這些嘗試會一起被保留下來，形成一個「解答集合」，方便後續進行比較與學習。

###2.1.1範例

假設有一個簡單的算術問題：

$$
q : \; \text{計算 } 2 + 2 \times 6
$$

輸出結果（當 $G = 8$ 時）：

$$
\{ o_1: 14 \; (\text{正確}),\; o_2: 16 \; (\text{錯誤}),\; o_3: 10 \; (\text{錯誤}),\; \ldots,\; o_8: 14 \; (\text{正確}) \}
$$

可以看到，有些答案正確（14），而有些答案則錯誤（16 或 10）。這種多樣性對接下來的步驟至關重要。

##2.2-優勢值計算（Advantage Calculation）：分辨誰更好

在我們已經得到多個回應之後，下一步就是要判斷：哪些答案比其他更好？GRPO 的簡潔性正是其真正亮點所在。與其他 RLHF 方法不同，傳統的 RLHF 通常要另外訓練一個獎勵模型，但 GRPO 的設計比較靈活，只要透過判斷品質的函數，計算出「優勢值（Advantage）」就可。

優勢值公式

GRPO 的關鍵洞察是：我們不需要絕對的品質分數，只要比較同一組裡的答案相對好壞即可。

這透過「標準化」來完成：

$$
A_i = \frac{r_i - \text{mean}(r_1, r_2, \dots, r_G)}{\text{std}(r_1, r_2, \dots, r_G)}
$$

其中：

$r_i$ 是單一回應的獎勵分數

$\text{mean}(\cdot)$ 是整組回應的平均獎勵

$\text{std}(\cdot)$ 是這組回應的標準差

引入獎勵分布

首先，為每個產生的回應分配一個獎勵分數。這裡我們以「獎勵模型（Reward Model, RM）」為例，但正如前一節提到的，我們其實可以使用任何能回傳獎勵的函數。

例如，根據答案是否正確來給分：若正確則 r_i = 1，若錯誤則 r_i = 0。接著，我們就可以利用這些分數來計算每個回應的優勢值。

###2.2.1-範例

延續前面的算術例子，假設我們有 8 個回應，其中 4 個正確、4 個錯誤。則：

| 指標       | 數值                                   |
| -------- | ------------------------------------ |
| 群組平均     | $\text{mean}(r_i) = 0.5$             |
| 標準差      | $\text{std}(r_i) = 0.53$             |
| 正確答案的優勢值 | $A_i = \frac{1 - 0.5}{0.53} = 0.94$  |
| 錯誤答案的優勢值 | $A_i = \frac{0 - 0.5}{0.53} = -0.94$ |


這個「優勢值」告訴我們每個答案相對於平均表現的好壞：

若 $A_i > 0$，代表該答案表現好於平均（高品質回應），模型在訓練時會提高它的生成機率。

若 $A_i < 0$，則代表答案低於平均（低品質回應），模型會降低它的生成機率。

在這個例子中，正確答案的 $A_i = 0.94$，因此在後續的最佳化過程中，它會被「強化」；而錯誤答案的 $A_i = -0.94$，則會被「削弱」。

📌 小結 透過優勢值計算，模型能在同一組輸出中辨別誰表現突出、誰表現落後，進而引導最佳化過程。接下來，我們就可以利用這些優勢值來更新策略。

##2.3-策略更新（Policy Update）：學習並持續改進

最後一步是 GRPO 訓練模型，就是利用前面計算出來的「優勢值」來更新模型，讓它在未來更有可能生成好的回應。這個過程既強大又穩定，主要基於兩個原則：

它鼓勵模型產生更多類似成功解決方案，同時擺脫效率較低的方法

它包含一個安全機制（稱為 KL 散度懲罰），可以防止模型一次發生太大的變化



---



# 第 三 章：環境建置與前置準備
在我們開始微調 LLM 之前，首先要確保我們的開發環境已經準備就緒。一個穩定且配置正確的環境是成功訓練模型的第一步。

> GPU 啟用： 請確保您的 Colab Notebook 已啟用 GPU。點擊菜單欄的 執行階段 (Runtime) -> 變更執行階段類型 (Change runtime type)，然後在 硬體加速器 (Hardware accelerator) 中選擇 GPU。

## 3.1  首先，來登入 HuggingFace

由於我們將從 Hugging Face hub 下載基礎模型 `unsloth/gemma-3-1b-it`，並將我們量化過的模型上傳回 Hugging Face hub，所以讓我們先登入 Hugging Face。

#### Google Colab 新功能
我將我的 Hugging Face token 存儲在左側的秘密標籤中。將我的 token 儲存在這個秘密標籤的好處是，我不會在筆記本中暴露 token，且我可以將這個秘密配置應用於我所有的 Colab 筆記本。

In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi

HF_TOKEN = userdata.get("HF_TOKEN")

api = HfApi(token=HF_TOKEN)
username = api.whoami()['name']
print(username)

Heng666


##3.2 必要 Python 套件安裝
無論您是使用 Conda 環境還是 Google Colab，接下來都需要安裝進行 LLM 微調所需的 Python 套件。請在您啟用的 Conda 環境中 (或 Colab 儲存格中) 執行以下指令：

*   transformers: Hugging Face 模型的基礎，
用於載入模型和分詞器。
*   datasets: Hugging Face 資料集函式庫，用於載入和處理資料。

In [ ]:
!pip install --quiet transformers datasets trl bitsandbytes peft trackio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.6/851.6 kB 53.7 MB/s eta 0:00:00


## 3.3 GPU 驅動與 CUDA 支援確認

LLM 的訓練需要大量的計算資源，幾乎必須仰賴 GPU (Graphics Processing Unit)。因此，確認您的環境是否正確偵測到 GPU 並支援 CUDA 是至關重要的一步。

CUDA 是 NVIDIA 提供的平行運算平台和程式設計模型，允許軟體使用 GPU 進行通用計算。PyTorch (一個流行的深度學習框架) 透過 CUDA 來利用 NVIDIA GPU 的運算能力。

請執行以下 Python 程式碼，檢查您的 PyTorch 環境是否已正確偵測到 CUDA：

In [ ]:
# 3.3.1 檢查 PyTorch 是否偵測到 CUDA
import torch

print(f"PyTorch 是否支援 CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"目前使用的 CUDA 裝置名稱: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 裝置數量: {torch.cuda.device_count()}")
    # 可以進一步檢查 CUDA 版本
    print(f"PyTorch 編譯的 CUDA 版本: {torch.version.cuda}")
    # 執行 nvidia-smi (僅限 Linux/Windows 終端機，Colab 可直接執行)
    # !nvidia-smi
else:
    print("警告：未偵測到 CUDA。模型訓練將在 CPU 上運行，速度會非常慢。")
    print("請檢查您的 GPU 驅動程式安裝、CUDA Toolkit 設定以及 PyTorch 的 CUDA 支援。")

PyTorch 是否支援 CUDA: True
目前使用的 CUDA 裝置名稱: Tesla T4
CUDA 裝置數量: 1
PyTorch 編譯的 CUDA 版本: 12.6


In [ ]:
!nvidia-smi

Tue Sep 16 15:05:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

> 【重要提醒】：如果 torch.cuda.is_available() 回傳 False，您可能需要：

確認您的電腦有 NVIDIA GPU。

安裝正確版本的 NVIDIA 顯示卡驅動程式。

安裝與您 PyTorch 版本相容的 CUDA Toolkit。

對於 Colab 用戶，請再次確認您已在執行階段中選擇了 GPU。

## 3.5 確認安裝套件版本

套入 GRPO 訓練, 模型參以及實驗追蹤。

In [ ]:
import trackio  # Experiment tracking dashboard
import re  # Regex patterns for reward functions

# GRPO training components
from trl import GRPOConfig, GRPOTrainer

# Model and tokenization
from transformers import (
    AutoModelForCausalLM,  # Causal language model loading
    AutoTokenizer,  # Text tokenization
)

# Dataset handling
from datasets import load_dataset

# Logging configuration
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Suppress httpx request logs that appear during trackio usage
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("gradio_client").setLevel(logging.WARNING)

#第四章、如何在有限資源下完成 GRPO 訓練？

Load up `Qwen/Qwen2.5-0.5B-Instruct`, and set parameters

## 4.0 仔入模型

選擇一個緊湊但功能強大的模型，適合具有記憶體限制的數學推理。

In [ ]:
# Select model optimized for instruction-following and reasoning
model_name = "Qwen/Qwen2.5-0.5B-Instruct"  # 3B parameter model balances capability and memory usage
max_seq_length = 2048  # Token limit for mathematical problems (reduce if OOM)

print(f"Loading model: {model_name}")
print(f"Max sequence length: {max_seq_length}")

Loading model: Qwen/Qwen2.5-0.5B-Instruct
Max sequence length: 2048


In [ ]:
# Load model with automatic device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Auto-distribute across available GPUs/CPU
    trust_remote_code=True,  # Allow custom model code execution
    torch_dtype=torch.float16,  # Use FP16 for non-quantized operations
)

# Load corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)  # Allow custom tokenizer code

# Ensure tokenizer has proper padding token for batch processing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Model loaded successfully!")
print(f"📊 Model parameters: ~{sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")
print(f"🧮 Quantized parameters: ~{sum(p.numel() for p in model.parameters() if hasattr(p, 'quant_type')) / 1e6:.1f}M")

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Model loaded successfully!
📊 Model parameters: ~494.0M
🧮 Quantized parameters: ~0.0M


## 4.3 GSM8K 資料集設定
<a name="Data"></a>

我們來設置具有結構化輸出格式的 GSM8K 數學推理資料集，以便逐步解決問題。這也是 Openai 最出名的資料集之ㄧ!

In [ ]:
from datasets import load_dataset
dataset = load_dataset("openai/gsm8k", "main", split = "train[:1000]")
dataset

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 1000
})

接著讓我們來檢視第一筆資料，看看結構之外，也看看問了什麼問題？

In [ ]:
dataset[0]["question"]

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [ ]:
dataset[0]["answer"]

'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'

我們可以注意到每一個答案都使用 `####` 符號表示，讓我們抽取。

In [ ]:
def extract_hash_answer(text):
    """Extract numerical answer from GSM8K format (#### marker)"""
    if "####" not in text: return None
    return text.split("####")[1].strip()
extract_hash_answer(dataset[0]["answer"])

'72'

我們現在建立一個可以自訂的系統提示。我們額外加入了 4 個符號，用於推理或思考階段，最後再加上一個最終答案：。

分別為
- reasoning_start = "<start_working_out>"
- reasoning_end   = "<end_working_out>"
- solution_start = "<SOLUTION>"
- solution_end = "</SOLUTION>"

In [ ]:
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""

system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'

讓我們來改照這個資料集！並查看第一列：

In [ ]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["question"]},
    ],
    "answer": extract_hash_answer(x["answer"]),
})
dataset[0]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': '72',
 'prompt': [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>',
   'role': 'system'},
  {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
   'role': 'user'}]}

In [ ]:
print(f"✅ 資料集已載入並處理完成！")
print(f"📊 訓練樣本數量: {len(dataset):,}")
print(f"🎯 範例問題: {dataset[0]['prompt'][1]['content']}...")
print(f"🎯 範例答案: {dataset[0]['answer']}")

# 顯示第一個樣本的結構以供驗證
print(f"\n📋 範例結構:")
print(f"   提示: {len(dataset[0]['prompt'])} 則訊息 (system + user)")
print(f"   答案: {dataset[0]['answer']} (獎勵用的標準答案)")


✅ Dataset loaded and processed!
📊 Training examples: 1,000
🎯 Sample question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?...
🎯 Sample answer: 72

📋 Example structure:
   Prompt: 2 messages (system + user)
   Answer: 72 (ground truth for rewards)


接著，我們建立一組正則表達式（regex）格式，用來比對推理區段與答案：

In [ ]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"  # Optional whitespace at start
    rf"{reasoning_start}.+?{reasoning_end}.*?"  # Reasoning section (non-greedy)
    rf"{solution_start}(.+?){solution_end}"  # Solution section with capture group
    rf"[\s]{{0,}}$",  # Optional whitespace at end
    flags=re.MULTILINE | re.DOTALL,  # Multi-line matching with . matching newlines
)

驗證這一段 Regex 可以正常運作：

In [ ]:
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<SOLUTION>2</SOLUTION>",
)

<re.Match object; span=(0, 71), match='<start_working_out>Let me think!<end_working_out>>



---



## 4.4 設計多道獎勵函數

實現四個互補的獎勵函數來評估數學推理的不同面向：

1. `match_format_exactly`:完美格式合規
2. `match_format_approximately`: 部分格式接近
3. `check_answer_correctness`：數學準確性，評分標準
4. `check_numbers_extraction`：能夠解析並輸出數字結果

接下來我們要建立一個獎勵函數，用來嚴格比對該格式——如果比對成功，就給予 3 分的獎勵：

In [ ]:
def match_format_exactly(completions, **kwargs):
    """
    High reward (3.0) for perfect format adherence
    Ensures model learns the complete structured output pattern
    """
    scores = []
    for completion in completions:
        response = completion[0]["content"]
        # Check if response matches complete format pattern
        score = 3.0 if match_format.search(response) is not None else 0.0
        scores.append(score)
    return scores

如果比對失敗，但模型至少部分遵循該格式，我們就按每個符號的匹配數來給予獎勵：

In [ ]:
def match_format_approximately(completions, **kwargs):
    """
    Graduated scoring for format elements
    Encourages learning individual components even if not perfect
    """
    scores = []
    for completion in completions:
        response = completion[0]["content"]
        score = 0

        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        # Award +0.5 for correct token count, -0.5 for wrong count
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end) == 1 else -0.5
        score += 0.5 if response.count(solution_start) == 1 else -0.5
        score += 0.5 if response.count(solution_end) == 1 else -0.5

        scores.append(score)
    return scores

最後，我們要擷取模型產生的答案，並對其進行獎勵或懲罰！同時，我們會根據答案與正確答案的接近程度（以比例計算）來給予獎勵：

In [ ]:
def check_answer_correctness(prompts, completions, answer, **kwargs):
    """
    Graduated scoring for mathematical accuracy:
    - 3.0: Exact match
    - 1.5: Within 10% (close answer)
    - 0.5: Within 20% (reasonable attempt)
    - -0.5: Wrong answer (penalty for incorrect math)
    """
    responses = [completion[0]["content"] for completion in completions]

    # Extract answers using format pattern
    extracted_responses = [
        guess.group(1) if (guess := match_format.search(r)) is not None else None for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:  # No extractable answer
            scores.append(0)
            continue

        # Exact string match gets full points
        if guess.strip() == true_answer.strip():
            scores.append(3.0)
        else:
            # Try numerical comparison for partial credit
            try:
                ratio = float(guess) / float(true_answer)
                if 0.9 <= ratio <= 1.1:  # Within 10%
                    scores.append(1.5)
                elif 0.8 <= ratio <= 1.2:  # Within 20%
                    scores.append(0.5)
                else:  # Wrong answer
                    scores.append(-0.5)
            except (ValueError, ZeroDivisionError):
                scores.append(-0.5)  # Invalid numerical format

    return scores

另外，有時候答案可能不只是一個數字，而是一句話，例如 'The solution is $20' ——我們就要從中擷取出 20。

In [ ]:
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})",  # Extract numbers from solution section
    flags=re.MULTILINE | re.DOTALL,  # Flexible pattern matching
)

match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>")

['0.34']

In [ ]:
def check_numbers_extraction(prompts, completions, answer, **kwargs):
    """
    Tests the model's ability to extract numerical values from solution sections
    Complementary to exact format matching - focuses on parsing capability
    """
    responses = [completion[0]["content"] for completion in completions]

    # Extract numbers from solution sections using number pattern
    extracted_responses = [
        guess.group(1) if (guess := match_numbers.search(r)) is not None else None for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:  # No extractable number
            scores.append(0)
            continue

        try:
            # Simple numerical equality check
            true_val = float(true_answer.strip())
            guess_val = float(guess.strip())
            # Binary scoring: correct (1.5) or incorrect (0)
            scores.append(1.5 if guess_val == true_val else 0.0)
        except (ValueError, TypeError):
            scores.append(0)  # Invalid number format

    return scores


---

<a name="Train"></a>
## 4.5 訓練模型

現在來設定 GRPO Trainer 與所有配置！

In [ ]:
# 設定 GRPO 訓練參數（用於數學推理）
training_args = GRPOConfig(
    # 適用於推理任務的學習參數
    learning_rate=5e-6,  # 保守的學習率，避免推理過程不穩定

    # 記憶體效率最佳化的批次設定
    per_device_train_batch_size=2,  # 小批次以符合 GPU 記憶體限制
    gradient_accumulation_steps=8,  # 有效批次大小 = 2 * 8 = 16

    # 數學問題的序列長度限制
    max_prompt_length=1024,  # 足以應付複雜的文字題
    max_completion_length=1024,  # 預留完整逐步推理的空間

    # 訓練時長與監控
    max_steps=10,  # 短時間展示用（正式訓練建議提升至 500+）
    logging_steps=1,  # 每步記錄一次指標，方便密切監控

    # 穩定性與輸出設定
    output_dir="./trl_grpo_outputs",
    max_grad_norm=0.1,  # 積極的梯度裁剪，確保訓練穩定
    report_to="trackio",  # 使用 trackio 追蹤實驗（替代 wandb/tensorboard）
)

### 4.5.2 使用 Trackio 整合進行訓練器初始化

使用我們的多獎勵系統和實驗追蹤來設定 GRPO 訓練器。

In [ ]:
# 建立帶有時間戳記的唯一執行名稱，確保追蹤資料新鮮
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_name = f"qwen2.5-0.5b-instruct-gsm8k-grpo-{timestamp}"

# 初始化 trackio 實驗追蹤，並指定唯一的執行名稱
trackio.init(
    project="GRPO-Mathematical-Reasoning",  # 專案名稱（用於組織與管理）
    name=run_name,  # 唯一執行識別碼（帶有時間戳記）
    config={
        # 模型與資料集設定
        "model_name": "Qwen/Qwen2.5-0.5B-Instruct",
        "dataset": "GSM8K",
        "technique": "GRPO + 4-bit",

        # 訓練超參數
        "learning_rate": training_args.learning_rate,
        "batch_size": training_args.per_device_train_batch_size,
        "gradient_accumulation_steps": training_args.gradient_accumulation_steps,
        "effective_batch_size": training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps,
        "max_steps": training_args.max_steps,

        # GRPO 特定設定
        "num_generations": training_args.num_generations,  # 預設：每步產生 8 個樣本
        "max_prompt_length": training_args.max_prompt_length,
        "max_completion_length": training_args.max_completion_length,

        # 獎勵系統
        "num_reward_functions": 4,
    },
)

print("🎯 GRPO 設定摘要:")
print(f"   學習率: {training_args.learning_rate}")
print(
    f"   有效批次大小: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}"
)
print(f"   訓練步數: {training_args.max_steps}")
print(f"   每步生成數: {training_args.num_generations}")
print(f"✅ Trackio 實驗追蹤初始化完成")
print(f"📊 執行名稱: {run_name}")

* Running on public URL: https://281d4fc2ce647119e4.gradio.live
* Trackio project initialized: GRPO-Mathematical-Reasoning
* Trackio metrics logged to: /root/.cache/huggingface/trackio
* View dashboard by running in your terminal:
trackio show --project "GRPO-Mathematical-Reasoning"
* or by running in Python: trackio.show(project="GRPO-Mathematical-Reasoning")
* Created new run: qwen2.5-0.5b-instruct-gsm8k-grpo-20250916-150726
🎯 GRPO Configuration Summary:
   Learning rate: 5e-06
   Effective batch size: 16
   Training steps: 10
   Generations per step: 8
✅ Trackio experiment tracking initialized
📊 Run name: qwen2.5-0.5b-instruct-gsm8k-grpo-20250916-150726


In [ ]:
# 初始化 GRPO 訓練器，啟用多重獎勵系統
# trackio_callback = TrackioCallback()  # 建立 trackio 日誌回呼函式

trainer = GRPOTrainer(
    model=model,  # 模型
    reward_funcs=[  # 四個互補的獎勵函數
        match_format_exactly,       # 完全符合結構
        match_format_approximately, # 部分格式符合給予部分分數
        check_answer_correctness,   # 數學正確性
        check_numbers_extraction,   # 數字解析能力
    ],
    args=training_args,  # 訓練參數設定
    train_dataset=dataset,  # 已處理好的 GSM8K 資料集
)

print("✅ GRPO 訓練器初始化成功！")
print(f"📊 訓練資料集: {len(dataset):,} 筆樣本")
print(f"🎯 啟用獎勵函數數量: {len(trainer.reward_funcs)} 個")
print(f"📈 Trackio 整合狀態: 已啟用")
print(f"🔄 準備開始訓練，每步生成 {training_args.num_generations} 個樣本")

✅ GRPO Trainer initialized successfully!
📊 Training dataset: 1,000 examples
🎯 Reward functions: 4 active
📈 Trackio integration: Enabled
🔄 Ready for training with 8 generations per step


### 4.5.3 開始 GRPO 培訓

從即時獎勵監控開始訓練過程。觀察格式合規性和數學準確性的逐步提升。


In [ ]:
# 執行 GRPO 訓練，並進行多重獎勵最佳化
print("🚀 開始 GRPO 訓練...")
print("📊 監控指標: 獎勵分數、KL 散度、策略梯度")
print("🔍 Trackio 將記錄: 損失、獎勵、學習率、梯度")

# 執行訓練流程
trainer.train()

# 完成 trackio 實驗
trackio.finish()

print("✅ 訓練成功完成！")
print(f"💾 模型已儲存至: {training_args.output_dir}")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


🚀 Starting GRPO training...
📊 Monitor metrics: reward scores, KL divergence, policy gradients
🔍 Trackio will log: losses, rewards, learning rate, gradients
* Trackio project initialized: huggingface
* Trackio metrics logged to: /root/.cache/huggingface/trackio
* View dashboard by running in your terminal:
trackio show --project "huggingface"
* or by running in Python: trackio.show(project="huggingface")
* Created new run: dainty-sunset-0


Step,Training Loss
1,0.230100
2,0.065200
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)
* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)
✅ Training completed successfully!
💾 Model saved to: ./trl_grpo_outputs


### 4.5.4 (選項)實驗儀表板

啟動互動式 trackio 儀表板來分析訓練進度、獎勵演進和模型效能指標。

In [ ]:
# Launch interactive trackio dashboard for experiment analysis
# View training curves, reward progression, loss evolution, and hyperparameter effects
trackio.show(project="GRPO-Mathematical-Reasoning")


# Alternative: Launch from command line with: trackio show --project "GRPO-Mathematical-Reasoning"

* Running on public URL: https://281d4fc2ce647119e4.gradio.live
* Trackio UI launched at: https://281d4fc2ce647119e4.gradio.live/?project=GRPO-Mathematical-Reasoning




---



## 4.6 保存模型

In [ ]:
# 假設 dpo_trainer 是您的 DPOTrainer 實例
output_dir = "./saved_grpo_model"  # 指定保存路徑

# 保存模型權重、配置和分詞器
trainer.save_model(output_dir)

print(f"模型已保存至 {output_dir}")

模型已保存至 ./saved_grpo_model


## 4.6 模型評估與測試

透過結構化輸出驗證來測試訓練模型的數學推理能力。

In [ ]:
# 定義模型測試函式，並設定最佳化生成參數
def test_model(question, max_length=512):
    """
    測試已訓練的模型在數學問題上的表現

    參數:
        question (str): 要解決的數學問題
        max_length (int): 最大生成的 token 數量

    回傳:
        str: 模型的結構化回覆（包含推理與解答）
    """
    # 使用對話範本格式化輸入
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    # 套用對話範本並轉換成文字
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,  # 加入 assistant 提示
        tokenize=False,  # 輸出字串而非 tokens
    )

    # 進行 tokenization 並移到對應的運算裝置
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    print(f"🤔 正在處理: {question}")

    # 使用最佳化的生成參數進行推理
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,  # 在創造力與一致性之間取得平衡
            do_sample=True,  # 啟用隨機取樣，探索多樣化推理路徑
            top_p=0.9,  # Nucleus sampling，提升生成品質
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,  # 降低重複推理步驟
            length_penalty=1.0,  # 對輸出長度保持中立偏好
            early_stopping=True,  # 在自然結束點提前停止
        )

    # 解碼並擷取生成的部分
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = response[len(text) :].strip()

    return generated_text

In [ ]:
# 在 GSM8K 題目上測試模型
gsm8k_question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"
expected_answer = "72"

# 生成模型回覆
gsm8k_response = test_model(gsm8k_question, max_length=768)

print(f"Question: {gsm8k_question}")
print(f"Model Response:\n{gsm8k_response}")

# 驗證格式是否符合要求
has_reasoning = reasoning_start in gsm8k_response and reasoning_end in gsm8k_response
has_solution = solution_start in gsm8k_response and solution_end in gsm8k_response

print(f"\n格式檢查:")
print(f"推理區段存在: {has_reasoning}")
print(f"解答區段存在: {has_solution}")

# 如果有解答區段，檢查答案是否正確
if has_solution:
    try:
        solution_text = gsm8k_response.split(solution_start)[1].split(solution_end)[0].strip()
        extracted_number = "".join(filter(str.isdigit, solution_text))
        expected_number = "".join(filter(str.isdigit, expected_answer))
        is_correct = extracted_number == expected_number

        print(f"擷取出的答案: {solution_text}")
        print(f"期望答案: {expected_answer}")
        print(f"是否正確: {is_correct}")
    except:
        print("無法擷取解答")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤔 Processing: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Model Response:
( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( < \( <

## 4.7 清理資源
釋放 GPU 記憶體並清除快取的張量，以實現最佳資源管理。



In [ ]:
from pathlib import Path

def remove_trackio_project(project_name):
    """刪除一個 trackio 專案，方式是移除其資料庫檔案"""
    cache_dir = Path.home() / ".cache" / "huggingface" / "trackio"
    db_file = cache_dir / f"{project_name}.db"

    if db_file.exists():
        db_file.unlink()
        print(f"已移除 trackio 專案: {project_name}")
    else:
        print(f"找不到專案: {project_name}")

In [ ]:
# 釋放 GPU 記憶體並清理 Python 垃圾回收
import gc

torch.cuda.empty_cache()  # 清除 PyTorch CUDA 記憶體快取
gc.collect()  # 執行 Python 垃圾回收器

print("✅ GPU 記憶體快取已清除")
print("✅ Python 垃圾回收已完成")
print("🧹 資源已釋放，可供其他程序使用")